In [ ]:
from pprint import pprint

In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
import resqpy.crs as rqcrs
import resqpy.grid as grr
import resqpy.model as rq
import resqpy.olio.vector_utilities as vec
from evo_schemas.components import AttributeDescription_V1_0_1 as AttributeDescription
from evo_schemas.components import NanContinuous_V1_0_1 as NanContinuous
from evo_schemas.components import VectorAttribute_V1_0_0 as VectorAttribute
from evo_schemas.elements import FloatArrayMd_V1_0_1 as FloatArrayMd
from resqpy.property import Property, PropertyCollection
from resqpy.surface import PointSet

from evo.data_converters.common import EvoWorkspaceMetadata, create_evo_object_service_and_data_client

In [ ]:
# Refresh model file each time
epc_file = "example.epc"
m = rq.new_model(epc_file)
m.store_epc()
model = rq.Model(epc_file)

# Create a CRS
crs = rqcrs.Crs(parent_model=model, epsg_code="4326", title="EPSG:4326")
crs.create_xml()

extent_kji = (2, 3, 4)
grid = grr.RegularGrid(
    model, extent_kji=extent_kji, origin=(0.0, 0.0, 1000.0), dxyz=(100.0, 100.0, -10.0), title="test grid"
)
grid.k_direction_is_down = False
grid.grid_is_right_handed = not grid.grid_is_right_handed
# generate xml and establish a property collection for the grid
grid.create_xml()
if grid.property_collection is None:
    grid.property_collection = PropertyCollection(support=grid)
pc = grid.property_collection

# Define shape to be the grid plus x,y,z points
points_shape = tuple(list(extent_kji) + [3])

# Create a static points property
stress = vec.unit_vectors(np.random.random(points_shape) + 0.1)
pc.add_cached_array_to_imported_list(
    cached_array=stress,
    source_info="random stress vectors",
    keyword="stress direction",
    uom="m",
    property_kind="length",
    indexable_element="cells",
    points=True,
)
pc.write_hdf5_for_imported_list()
pc.create_xml_for_imported_list_and_add_parts_to_model()

model.store_epc()
model.h5_release()

In [ ]:
# Convert a Points Property into a Evo VectorAttribute


def convert_points_property(p: Property, data_client, idx_valid=None) -> VectorAttribute:
    """Converts a RESQML Points Property object to a VectorAttribute object.
    Args:
        p (Property): The RESQML Property object.
        data_client (ObjectDataClient): The ObjectDataClient object.
        idx_valid (np.ndarray): Optional. The indices of the valid values in the property array.
    """
    go = None

    if p.is_points:
        array_values = p.array_ref(masked=True)[idx_valid] if idx_valid is not None else p.array_ref()
        xyz_array = array_values.reshape(-1, 3)
        go = _convert_points_to_vector_attribute(p.title, str(p.uuid), xyz_array, data_client)

    return go


def convert_resqml_pointset(ps: PointSet, data_client, idx_valid=None) -> VectorAttribute:
    """Converts a RESQML Pointset object to a VectorAttribute object.
    Args:
        ps (PointSet): The RESQML PointSet object.
        data_client (ObjectDataClient): The ObjectDataClient object.
        idx_valid (np.ndarray): Optional. The indices of the valid values in the property array.
    """
    xyz_array = ps.full_array_ref(masked=True)[idx_valid] if idx_valid is not None else ps.full_array_ref()

    return _convert_points_to_vector_attribute(ps.title, str(ps.uuid), xyz_array, data_client)


def _convert_points_to_vector_attribute(name: str, key: str, array_values: list, data_client) -> VectorAttribute:
    """
    Converts a list of x, y, z coordinates to a VectorAttribute object. The list of coordinates is reshaped to a 2D
    array with 3 columns (x, y, z). The name and key are used to create the VectorAttribute object.
    Args:
        name (str): The name of the VectorAttribute object.
        key (str): The unique key of the VectorAttribute object.
        array_values (list): The list of x, y, z coordinates.
        data_client (ObjectDataClient): The ObjectDataClient object.
    """
    xyz_array = array_values.reshape((array_values.size // 3, 3))
    df = pd.DataFrame(xyz_array, columns=["x", "y", "z"])
    schema = pa.schema([("x", pa.float64()), ("y", pa.float64()), ("z", pa.float64())])
    table = pa.Table.from_pandas(df, schema=schema)
    float_array_args = data_client.save_table(table)
    float_array_go = FloatArrayMd.from_dict(float_array_args)

    return VectorAttribute(
        name=name,
        key=key,
        attribute_description=AttributeDescription(discipline="None", type="ContinuousProperty"),
        nan_description=NanContinuous(values=[]),
        values=float_array_go,
    )

In [ ]:
# Acquire the data_client
meta_data = EvoWorkspaceMetadata(
    org_id="8ac3f041-b186-41f9-84ba-43d60f8683be", workspace_id="2cf1697f-2771-485e-848d-e6674d2ac63f"
)
_, data_client = create_evo_object_service_and_data_client(meta_data)

# This method finds the PointsProperty in the model by object type and converts it
# We are only expecting one of these in this example
uuid = model.uuids(obj_type="PointsProperty")[0]
property = Property(parent_model=model, uuid=uuid)
print(f"\nPoints Property from model: {property.title}")
pprint(convert_points_property(property, data_client=data_client))

# Or, find the part in the PropertyCollection of the support object, the grid
# and create a Points Property from that and convert it.
points_part = pc.singleton(citation_title="stress direction", points=True)
stress_uuid = pc.uuid_for_part(points_part)
property = Property(model, uuid=stress_uuid)
print(f"\nPoints Property from support object property collection: {property.title}")
pprint(convert_points_property(property, data_client=data_client))